# All pairs shortest path
The all pairs shortest paths (APSP) algorithm finds the lengths (summed weights) of the shortest paths between all pairs of vertices such that the sum of the weights of the path edges is minimized.

APSP was added in MADlib 1.12.

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum 4.3.10.0
%sql postgresql://gpdbchina@10.194.10.68:61000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum 4.2.3.0
#%sql postgresql://gpdbchina@10.194.10.68:55000/madlib

u'Connected: gpdbchina@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.12-dev, git revision: rel/v1.11-16-gf50f76d, cmake configuration time: Mon Jun 12 20:12:42 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


# 1.  Create vertex and edge tables

In [4]:
%%sql 
DROP TABLE IF EXISTS vertex, edge;

CREATE TABLE vertex(
        id INTEGER
        );

CREATE TABLE edge(
        src INTEGER,
        dest INTEGER,
        weight FLOAT8
        );

INSERT INTO vertex VALUES
(0),
(1),
(2),
(3),
(4),
(5),
(6),
(7);

INSERT INTO edge VALUES
(0, 1, 1.0),
(0, 2, 1.0),
(0, 4, 10.0),
(1, 2, 2.0),
(1, 3, 10.0),
(2, 3, 1.0),
(2, 5, 1.0),
(2, 6, 3.0),
(3, 0, 1.0),
(4, 0, -2.0),
(5, 6, 1.0),
(6, 7, 1.0);

SELECT * FROM edge ORDER BY src, dest;

Done.
Done.
Done.
8 rows affected.
12 rows affected.
12 rows affected.


src,dest,weight
0,1,1.0
0,2,1.0
0,4,10.0
1,2,2.0
1,3,10.0
2,3,1.0
2,5,1.0
2,6,3.0
3,0,1.0
4,0,-2.0


# 2.  Calculate the shortest paths

In [5]:
%%sql
DROP TABLE IF EXISTS out, out_summary;

SELECT madlib.graph_apsp(
                         'vertex',      -- Vertex table
                         NULL,          -- Vertix id column (NULL means use default naming)
                         'edge',        -- Edge table
                         NULL,          -- Edge arguments (NULL means use default naming)
                         'out');        -- Output table of shortest paths

SELECT * FROM out ORDER BY src,dest;

Done.
1 rows affected.
64 rows affected.


src,dest,weight,parent
0,0,0.0,0
0,1,1.0,1
0,2,1.0,2
0,3,2.0,2
0,4,10.0,4
0,5,2.0,2
0,6,3.0,5
0,7,4.0,6
1,0,4.0,3
1,1,0.0,1


# 3.  Get the shortest path from vertex 0 to vertex 5

In [11]:
%%sql
DROP TABLE IF EXISTS out_path;
SELECT madlib.graph_apsp_get_path('out',0,5,'out_path');
SELECT * FROM out_path;

Done.
1 rows affected.
1 rows affected.


path
"[0, 2, 5]"


# 4.  Custom column names
Now let's do a similar example except using different column names in the tables (i.e., not the defaults). Create the vertex and edge tables:

In [12]:
%%sql 
DROP TABLE IF EXISTS vertex_alt, edge_alt;
CREATE TABLE vertex_alt AS SELECT id AS v_id FROM vertex;
CREATE TABLE edge_alt AS SELECT src AS e_src, dest, weight AS e_weight FROM edge;
SELECT * FROM edge_alt ORDER BY e_src, dest;

Done.
8 rows affected.
12 rows affected.
12 rows affected.


e_src,dest,e_weight
0,1,1.0
0,2,1.0
0,4,10.0
1,2,2.0
1,3,10.0
2,3,1.0
2,5,1.0
2,6,3.0
3,0,1.0
4,0,-2.0


# 5.  Calculate the shortest paths

In [13]:
%%sql
DROP TABLE IF EXISTS out_alt, out_alt_summary;

SELECT madlib.graph_apsp(
                         'vertex_alt',                  -- Vertex table
                         'v_id',                        -- Vertex id column (NULL means use default naming)
                         'edge_alt',                    -- Edge table
                         'src=e_src, weight=e_weight',  -- Edge arguments (NULL means use default naming)
                         'out_alt');                    -- Output table of shortest paths

SELECT * FROM out_alt ORDER BY e_src, dest;

Done.
1 rows affected.
64 rows affected.


e_src,dest,e_weight,parent
0,0,0.0,0
0,1,1.0,1
0,2,1.0,2
0,3,2.0,2
0,4,10.0,4
0,5,2.0,2
0,6,3.0,5
0,7,4.0,6
1,0,4.0,3
1,1,0.0,1


# 6. Grouping
Create a graph with 2 groups and find APSP for each group:

In [17]:
%%sql
DROP TABLE IF EXISTS edge_gr;

CREATE TABLE edge_gr AS
(
  SELECT *, 0 AS grp FROM edge
  UNION
  SELECT *, 1 AS grp FROM edge WHERE src < 6 AND dest < 6
);

INSERT INTO edge_gr VALUES
(4,5,-20,1);

SELECT * FROM edge_gr ORDER BY grp, src, dest;

Done.
21 rows affected.
1 rows affected.
22 rows affected.


src,dest,weight,grp
0,1,1.0,0
0,2,1.0,0
0,4,10.0,0
1,2,2.0,0
1,3,10.0,0
2,3,1.0,0
2,5,1.0,0
2,6,3.0,0
3,0,1.0,0
4,0,-2.0,0


# 7. Find APSP for all groups

In [15]:
%%sql
DROP TABLE IF EXISTS out_gr, out_gr_summary;

SELECT madlib.graph_apsp(
                         'vertex',      -- Vertex table
                         NULL,          -- Vertex id column (NULL means use default naming)
                         'edge_gr',     -- Edge table
                         NULL,          -- Edge arguments (NULL means use default naming)
                         'out_gr',      -- Output table of shortest paths
                         'grp'          -- Grouping columns
);

SELECT * FROM out_gr WHERE src < 2 ORDER BY grp,src,dest;

Done.
1 rows affected.
28 rows affected.


grp,src,dest,weight,parent
0,0,0,0.0,0
0,0,1,1.0,1
0,0,2,1.0,2
0,0,3,2.0,2
0,0,4,10.0,4
0,0,5,2.0,2
0,0,6,3.0,5
0,0,7,4.0,6
0,1,0,4.0,3
0,1,1,0.0,1


# 8. Find the path from vertex 0 to vertex 5 in every group

In [16]:
%%sql
DROP TABLE IF EXISTS out_gr_path;
SELECT madlib.graph_apsp_get_path('out_gr',0,5,'out_gr_path');
SELECT * FROM out_gr_path ORDER BY grp;

Done.
1 rows affected.
2 rows affected.


grp,path
0,"[0, 2, 5]"
1,"[0, 4, 5]"
